In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
item_list31 = [
        "工业品","医药","酒类","家庭清洁/纸品","手机通讯","家装建材","宠物生活","服饰内衣","图书","母婴","数码",
        "生鲜","食品饮料","家具","运动户外","鞋靴","电脑、办公","家用电器","二手商品","汽车用品","礼品","美妆护肤",
        "家纺","农资园艺","厨具","珠宝首饰","玩具乐器","家居日用","医疗保健","箱包皮具","个人护理"]
# 去掉（二手商品，礼品，工业品，医药，食品饮料），合并（家庭清洁-纸品，家居日用）

In [ ]:
df_covidtime_cityclass = pd.read_csv('./data/covidtime_g14_nooverlap.csv')
df_covidtime_cityclass['start_dt'] = pd.to_datetime(df_covidtime_cityclass['start_dt'])
df_covidtime_cityclass['end_dt'] = pd.to_datetime(df_covidtime_cityclass['end_dt'])
df_covidtime_cityclass.tail(5)

In [ ]:
df_cityfeatures = pd.read_csv('./data/city_time_gdp_grouped_data_3.csv')
df_cityfeatures = df_cityfeatures.rename(columns={'day': 'sale_ord_dt'})
df_cityfeatures = df_cityfeatures.dropna().sort_values(['city_ch_name', 'sale_ord_dt']).reset_index(drop=True)
df_cityfeatures

In [ ]:
for year in [2020, 2021, 2022]:
    df = pd.read_csv(f'./data/data_{year}_city_cityclass_1.csv')
    df = df.groupby(['city_ch_name', 'item_first_cate_name', 'sale_ord_dt'], as_index=False).agg({
        'nums': 'sum',
        'province_code': 'first',
        'city_class': 'first',
    })
    df = pd.merge(df, df_cityfeatures, on=['city_ch_name', 'sale_ord_dt'], how='left').dropna()
    df.to_csv(f'./data/data_{year}_allcate_allday.csv', index=False, encoding='utf-8-sig')

In [ ]:
df = pd.read_csv(f'./data/data_2023_city_cityclass_1.csv')
df = df.groupby(['city_ch_name', 'item_first_cate_name', 'sale_ord_dt'], as_index=False).agg({
    'nums': 'sum',
    'province_code': 'first',
    'city_class': 'first',
})
df.to_csv(f'./data/data_2023_allcate_allday.csv', index=False, encoding='utf-8-sig')

In [ ]:
df1 = pd.read_csv('./data/data_2020_allcate_allday.csv')
df2 = pd.read_csv('./data/data_2021_allcate_allday.csv')
df3 = pd.read_csv('./data/data_2022_allcate_allday.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)
df['sale_ord_dt'] = pd.to_datetime(df['sale_ord_dt'])
df.to_csv('./data/data_2020-2022_allcate_allday.csv', index=False, encoding='utf-8-sig')
df

## 取封控开始的前后14天作数据集

In [ ]:
item_list31 = [
        "工业品","医药","酒类","家庭清洁/纸品","手机通讯","家装建材","宠物生活","服饰内衣","图书","母婴","数码",
        "生鲜","食品饮料","家具","运动户外","鞋靴","电脑、办公","家用电器","二手商品","汽车用品","礼品","美妆护肤",
        "家纺","农资园艺","厨具","珠宝首饰","玩具乐器","家居日用","医疗保健","箱包皮具","个人护理"]
# 去掉（二手商品，礼品，工业品，医药，食品饮料），合并（家庭清洁-纸品，家居日用）

In [ ]:
df = pd.read_csv('./data/data_2020-2022_allcate_allday.csv', parse_dates=['sale_ord_dt'])
df

In [ ]:
# 都有的城市
#df = pd.read_csv(f'./data/data_2020-2022_allcate_allday.csv')
a1 = df_covidtime_cityclass['city_ch_name'].unique()
a2 = df['city_ch_name'].unique()
print(np.setdiff1d(a1, a2), np.setdiff1d(a2, a1))
len(a1), a1, "-"*100, len(a2), a2

In [ ]:
intersection_city = np.intersect1d(a1, a2)
len(intersection_city)

In [ ]:
# 2020-2022
results = []
df_miss = pd.DataFrame(columns=['city', 'item', 'nannum', 'daynum'])
for index, row in df_covidtime_cityclass.iterrows():
    city = row['city_ch_name']
    s_date = row['start_dt']
    if city in intersection_city:
        for item in item_list31:
            # 计算日期范围
            start_date = s_date - pd.Timedelta(days=14)
            end_date = s_date + pd.Timedelta(days=13)
            
            # 筛选符合条件的行
            filtered = df[(df['city_ch_name'] == city) & (df['item_first_cate_name'] == item) & (df['sale_ord_dt'] >= start_date) & (df['sale_ord_dt'] <= end_date)]
            
            # 生成日期范围
            date_range = pd.date_range(start=start_date, end=end_date)
            
            # 重新索引以填充缺失日期为 NaN
            filtered = filtered.set_index('sale_ord_dt').reindex(date_range).reset_index()
            filtered['city_ch_name'] = city
            filtered['nums'] = filtered['nums']#.fillna(np.nan)
            
            # 重命名列
            filtered.rename(columns={'index': 'sale_ord_dt'}, inplace=True)
            df_miss.loc[len(df_miss)] = [city, item, filtered.isna().any(axis=1).sum(), len(filtered)]
            if filtered.isna().any(axis=1).sum() / len(filtered) > 0.1:
                print(city, item, filtered.isna().any(axis=1).sum(), len(filtered))
            # 添加到结果列表
            results.append(filtered)

df_miss.to_csv('./data/data_2020-2022_28days_31items_nannum.csv', index=False, encoding='utf-8-sig')
# 合并所有结果
final_result = pd.concat(results, ignore_index=True)
final_result.to_csv('./data/data_2020-2022_28days_31items.csv', index=False, encoding='utf-8-sig')
final_result

In [ ]:
# 2023取对应日期的数据
df_covidtime_cityclass['start_dt'] = pd.to_datetime(df_covidtime_cityclass['start_dt'])
df_covidtime_cityclass['end_dt'] = pd.to_datetime(df_covidtime_cityclass['end_dt'])

df_time2023 = df_covidtime_cityclass.copy()
# 检查年份是否相同
same_year = df_time2023['start_dt'].dt.year == df_time2023['end_dt'].dt.year

# 修改年份为2023
df_time2023.loc[same_year, 'start_dt'] = df_time2023.loc[same_year, 'start_dt'].apply(lambda x: x.replace(year=2023))
df_time2023.loc[same_year, 'end_dt'] = df_time2023.loc[same_year, 'end_dt'].apply(lambda x: x.replace(year=2023))

# 130           大连 2020-12-15 2021-01-24       二线城市      40
# 260           沈阳 2020-12-23 2021-01-20      新一线城市      28
# 385           西安 2021-12-09 2022-01-05      新一线城市      27
# 年份不同的行
df_time2023.loc[~same_year, 'start_dt'] = df_time2023.loc[~same_year, 'start_dt'].apply(lambda x: x.replace(year=2022))
df_time2023.loc[~same_year, 'end_dt'] = df_time2023.loc[~same_year, 'end_dt'].apply(lambda x: x.replace(year=2023))
df_time2023['period'] = (df_time2023['end_dt'] - df_time2023['start_dt']).dt.days
df_time2023.to_csv('./data/time_2023.csv', index=False, encoding='utf-8-sig')
df_time2023

In [ ]:
df = pd.read_csv('./data/data_2023_allcate_allday.csv', parse_dates=['sale_ord_dt'])
df.tail(5)

In [ ]:
results = []
df_miss = pd.DataFrame(columns=['city', 'item', 'nannum', 'daynum'])
for index, row in df_time2023.iterrows():
    city = row['city_ch_name']
    s_date = row['start_dt']
    if city in intersection_city:
        for item in item_list31:
            # 计算日期范围
            start_date = s_date - pd.Timedelta(days=14)
            end_date = s_date + pd.Timedelta(days=13)
            
            # 筛选符合条件的行
            filtered = df[(df['city_ch_name'] == city) & (df['item_first_cate_name'] == item) & (df['sale_ord_dt'] >= start_date) & (df['sale_ord_dt'] <= end_date)]
            
            # 生成日期范围
            date_range = pd.date_range(start=start_date, end=end_date)
            
            # 重新索引以填充缺失日期为 NaN
            filtered = filtered.set_index('sale_ord_dt').reindex(date_range).reset_index()
            filtered['city_ch_name'] = city
            filtered['nums'] = filtered['nums']#.fillna(np.nan)
            
            # 重命名列
            filtered.rename(columns={'index': 'sale_ord_dt'}, inplace=True)
            df_miss.loc[len(df_miss)] = [city, item, filtered.isna().any(axis=1).sum(), len(filtered)]
            if filtered.isna().any(axis=1).sum() / len(filtered) > 0.1:
                print(city, item, filtered.isna().any(axis=1).sum(), len(filtered))
            # 添加到结果列表
            results.append(filtered)

df_miss.to_csv('./data/data_2023_28days_31items_nannum.csv', index=False, encoding='utf-8-sig')
# 合并所有结果
final_result2 = pd.concat(results, ignore_index=True)
final_result2.to_csv('./data/data_2023_28days_31items.csv', index=False, encoding='utf-8-sig')
final_result2

### NAN数量多于3的去掉

In [ ]:
df_miss1 = pd.read_csv('./data/data_2020-2022_28days_31items_nannum.csv')
df_miss2 = pd.read_csv('./data/data_2023_28days_31items_nannum.csv')

In [ ]:
df_20202022_28days_31items = pd.read_csv('./data/data_2020-2022_28days_31items.csv')
df_2023_28days_31items = pd.read_csv('./data/data_2023_28days_31items.csv')

###### item列填上

In [ ]:
# 获取 DataFrame 的总行数
num_rows = len(df_20202022_28days_31items)

# 记录冲突的索引
conflicting_indices = []

# 遍历每组 28 行
for i, start_index in enumerate(range(0, num_rows, 28)):
    end_index = min(start_index + 28, num_rows)
    
    # 获取当前组
    group = df_20202022_28days_31items.iloc[start_index:end_index]
    
    # 计算当前组要填充的值
    fill_value = item_list31[i % len(item_list31)]
    
    # 检查冲突并填充
    for index, row in group.iterrows():
        original_value = row['item_first_cate_name']
        if pd.notna(original_value) and original_value != fill_value:
            conflicting_indices.append(index)
        else:
            df_20202022_28days_31items.at[index, 'item_first_cate_name'] = fill_value

# 打印冲突的行索引
if conflicting_indices:
    print("冲突的行索引:", conflicting_indices)
else:
    print("没有发现冲突")
    df_20202022_28days_31items.to_csv('./data/data_2020-2022_28days_31items.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 获取 DataFrame 的总行数
num_rows = len(df_2023_28days_31items)

# 记录冲突的索引
conflicting_indices = []

# 遍历每组 28 行
for i, start_index in enumerate(range(0, num_rows, 28)):
    end_index = min(start_index + 28, num_rows)
    
    # 获取当前组
    group = df_2023_28days_31items.iloc[start_index:end_index]
    
    # 计算当前组要填充的值
    fill_value = item_list31[i % len(item_list31)]
    
    # 检查冲突并填充
    for index, row in group.iterrows():
        original_value = row['item_first_cate_name']
        if pd.notna(original_value) and original_value != fill_value:
            conflicting_indices.append(index)
        else:
            df_2023_28days_31items.at[index, 'item_first_cate_name'] = fill_value

# 打印冲突的行索引
if conflicting_indices:
    print("冲突的行索引:", conflicting_indices)
else:
    print("没有发现冲突")
    df_2023_28days_31items.to_csv('./data/data_2023_28days_31items.csv', index=False, encoding='utf-8-sig')

###### 取出NA少于3的

In [ ]:
df_miss1 = df_miss1[df_miss1['nannum'] > 3][['city', 'item']]
df_miss2 = df_miss2[df_miss2['nannum'] > 3][['city', 'item']]
df = pd.concat([df_miss1, df_miss2]).rename(columns={'city': 'city_ch_name', 'item': 'item_first_cate_name'}).drop_duplicates()

In [ ]:
df

In [ ]:
df_20202022_fewnan = df_20202022_28days_31items.merge(df, on=['city_ch_name', 'item_first_cate_name'], how='left', indicator=True)
df_20202022_fewnan = df_20202022_fewnan[df_20202022_fewnan['_merge'] == 'left_only'].drop(columns=['_merge']).reset_index(drop=True)
df_20202022_fewnan.to_csv('./data/data_2020-2022_fewnan.csv', index=False, encoding='utf-8-sig')
df_20202022_fewnan

In [ ]:
df_2023_fewnan = df_2023_28days_31items.merge(df, on=['city_ch_name', 'item_first_cate_name'], how='left', indicator=True)
df_2023_fewnan = df_2023_fewnan[df_2023_fewnan['_merge'] == 'left_only'].drop(columns=['_merge']).reset_index(drop=True)
df_2023_fewnan.to_csv('./data/data_2023_fewnan.csv', index=False, encoding='utf-8-sig')
df_2023_fewnan

###### province_code city_class gdp	longitude latitude列填上

In [ ]:
# 2020-2022
# 填NAN
df = df_20202022_fewnan
# 将每28行分为一组
df['group'] = df.index // 28

# 定义需要填充的列
cols_to_fill = ['province_code', 'city_class', 'gdp', 'longitude', 'latitude']

# 使用 groupby 和 transform 填充缺失值
for col in cols_to_fill:
    # def fill_missing(x):
    #     mode_val = x.mode()
    #     if not mode_val.empty:
    #         return x.fillna(mode_val.iloc[0])
    #     else:
    #         print(x)
    #         return x

    # df[col] = df.groupby('group')[col].transform(fill_missing)
    df[col] = df.groupby('group')[col].transform(lambda x: x.fillna(x.mode().iloc[0]))

# 删除辅助列
df.drop(columns='group', inplace=True)
df.to_csv('./data/data_2020-2022_fewnan.csv', index=False, encoding='utf-8-sig')

###### year	month mon_day

In [ ]:
df = df_20202022_fewnan
# 找出包含 NA 的行
df['sale_ord_dt'] = pd.to_datetime(df['sale_ord_dt'])
na_rows = df[df.isna().any(axis=1)]

# 提取日期信息并填充到年、月、日列
df.loc[na_rows.index, 'year'] = na_rows['sale_ord_dt'].dt.year
df.loc[na_rows.index, 'month'] = na_rows['sale_ord_dt'].dt.month
df.loc[na_rows.index, 'mon_day'] = na_rows['sale_ord_dt'].dt.day
df.to_csv('./data/data_2020-2022_fewnan.csv', index=False, encoding='utf-8-sig')

In [ ]:
df = df_2023_fewnan
df['sale_ord_dt'] = pd.to_datetime(df['sale_ord_dt'])

# 提取日期信息并填充到年、月、日列
df['year'] = df['sale_ord_dt'].dt.year
df['month'] = df['sale_ord_dt'].dt.month
df['mon_day'] = df['sale_ord_dt'].dt.day
df.to_csv('./data/data_2023_fewnan.csv', index=False, encoding='utf-8-sig')

In [ ]:
df_20202022_fewnan.isna().sum(), df_2023_fewnan.isna().sum()

### 缺失的品类多的去掉，item26

In [ ]:
array=np.array(df_miss1['item'])
values, counts = np.unique(array, return_counts=True)
value_counts = dict(zip(values, counts))
print(value_counts)

In [ ]:
df = pd.read_csv('./data/data_2020-2022_28days_31items.csv')
df

In [ ]:
items = ['二手商品', '农资园艺', '医药', '工业品', '礼品']#
df_20202022_26 = df[~df['item_first_cate_name'].isin(items)].reset_index(drop=True)
print(df_20202022_26.isna().sum())
df_20202022_26.to_csv('./data/data_2020-2022_28days_26items.csv', index=False, encoding='utf-8-sig')
df_20202022_26